In [1]:
import plotly.graph_objects as go
import networkx as nx
import os

# reading input log file

# test_name = "arldm_chunk2k4c_ssd"
# test_name = "vist_1t"
# test_name = "vist_1t_chunk"
# test_name = "ss_s9f9p8_0"
test_name = "s9f9p8_0"
# test_name = "ddmd_skipsim"
# test_name = "ddmd"
# test_name = "64_ds200_p1_r32_t2"
# test_name = "f24s9p12_1"
# test_name = "sync-write-3d-contig-contig-read-1024r_t3"

stat_path=f"example_stat/{test_name}"
image_path=f"{stat_path}/images"

VFD_ACCESS_SKIP=10

ADD_ADDR=False

STAGE_START = 0
STAGE_END = 9

SUBGRAPH=False


In [2]:
# My utility functions
import utils.stat_loader as sload
import utils.stat_print as sp
import utils.vfd_stat2graph as vfd2g
import utils.vfd_graph2sankey as vfd2sk
import utils.full_stat2graph as f2g

# Improve Functions
## TODO
- Add dataset nodes
- Add file address ordering nodes


In [3]:
TASK_ORDER_LIST = sload.load_task_order_list(stat_path)
STAGE_END = sload.correct_end_stage(TASK_ORDER_LIST, STAGE_END)

TASK_ORDER_LIST = sload.current_task_order_list(TASK_ORDER_LIST, STAGE_START, STAGE_END)

TASK_LISTS = list(TASK_ORDER_LIST.keys())

print(f"TASK_ORDER_LIST = {TASK_ORDER_LIST}")
TASK_LISTS


STAGE_END is not in TASK_ORDER_LIST, set to max order: 8
TASK_ORDER_LIST = {'run_idfeature': 0, 'run_tracksingle': 1, 'run_gettracks': 2, 'run_trackstats': 3, 'run_identifymcs': 4, 'run_matchpf': 5, 'run_robustmcs': 6, 'run_mapfeature': 7, 'run_speed': 8}


['run_idfeature',
 'run_tracksingle',
 'run_gettracks',
 'run_trackstats',
 'run_identifymcs',
 'run_matchpf',
 'run_robustmcs',
 'run_mapfeature',
 'run_speed']

In [4]:
vfd_files = sload.find_files_with_pattern(stat_path, "vfd")
# vfd_files = vfd_files[0:1]
print(f"vfd_files: {vfd_files}")

vfd_dict = sload.load_stat_json(vfd_files)
# print(vfd_dict)


vol_files = sload.find_files_with_pattern(stat_path, "vol")
# vol_files = vol_files[0:1]
print(f"vol_files: {vol_files}")

vol_dict = sload.load_stat_json(vol_files)

print("loading json done")

vfd_files: ['example_stat/s9f9p8_0/71911-vfd_data_stat.json', 'example_stat/s9f9p8_0/71974-vfd_data_stat.json', 'example_stat/s9f9p8_0/71970-vfd_data_stat.json', 'example_stat/s9f9p8_0/71964-vfd_data_stat.json', 'example_stat/s9f9p8_0/71959-vfd_data_stat.json', 'example_stat/s9f9p8_0/71966-vfd_data_stat.json', 'example_stat/s9f9p8_0/71977-vfd_data_stat.json', 'example_stat/s9f9p8_0/71961-vfd_data_stat.json', 'example_stat/s9f9p8_0/71969-vfd_data_stat.json']
loading example_stat/s9f9p8_0/71911-vfd_data_stat.json
loading example_stat/s9f9p8_0/71974-vfd_data_stat.json
loading example_stat/s9f9p8_0/71970-vfd_data_stat.json
loading example_stat/s9f9p8_0/71964-vfd_data_stat.json
loading example_stat/s9f9p8_0/71959-vfd_data_stat.json
loading example_stat/s9f9p8_0/71966-vfd_data_stat.json
loading example_stat/s9f9p8_0/71977-vfd_data_stat.json
loading example_stat/s9f9p8_0/71961-vfd_data_stat.json
loading example_stat/s9f9p8_0/71969-vfd_data_stat.json
vol_files: ['example_stat/s9f9p8_0/71911-vo

In [5]:
# # Show VFD Tracker overhead
# sp.show_all_overhead("VFD", vfd_dict)

In [6]:
def inc_in_dict(dic, k):
    if k not in dic:
        dic[k] = 0
    else:
        dic[k]+=1



G_VFD = nx.DiGraph()
G_VFD = f2g.add_task_dset_file_nodes(G_VFD, vfd_dict, TASK_ORDER_LIST, add_addr=ADD_ADDR)

# sp.display_all_edges_attr(G_VFD)
# sp.display_all_nodes_position(G_VFD)

In [7]:

# file_page_nodes_attr, dset_page_edges = f2g.get_file_dset_maps(vfd_dict, TASK_ORDER_LIST)

file_page_nodes_attr, dset_page_edges = f2g.parallel_get_file_dset_maps(vfd_dict, TASK_ORDER_LIST)


# for k,v in file_page_nodes_attr.items(): print(f"file_page_nodes_attr: {k} -> {v}")
# for k,v in add_edge_stat.items(): print(f"add_edge_stat: {k} -> {v}")



file_page_nodes_attr: {'[0-76)': {'pos': (0, 0), 'rpos': 0, 'range': (0, 76), 'size': 4980736}, '[76-152)': {'pos': (0, 1), 'rpos': 0, 'range': (76, 152), 'size': 4980736}, '[152-228)': {'pos': (0, 2), 'rpos': 0, 'range': (152, 228), 'size': 4980736}, '[228-304)': {'pos': (0, 3), 'rpos': 0, 'range': (228, 304), 'size': 4980736}}
dset_page_edges: {('[0-76)', '-0-R'): {'access_cnt': 132, 'access_type': 'read'}, ('-0-W', '[0-76)'): {'access_cnt': 486, 'access_type': 'write'}, ('-0-W', '[76-152)'): {'access_cnt': 126, 'access_type': 'write'}, ('-0-W', '[152-228)'): {'access_cnt': 19, 'access_type': 'write'}, ('[0-76)', 'base_time-0-R'): {'access_cnt': 108, 'access_type': 'read'}, ('base_time-0-W', '[0-76)'): {'access_cnt': 18, 'access_type': 'write'}, ('cloudnumber-0-W', '[76-152)'): {'access_cnt': 203, 'access_type': 'write'}, ('cloudnumber_orig-0-W', '[76-152)'): {'access_cnt': 768, 'access_type': 'write'}, ('cloudnumber_orig-0-W', '[152-228)'): {'access_cnt': 531, 'access_type': 'write'

In [8]:
def update_nodes_edges(G,add_edge_stat, edges_to_remove, nodes_to_add):
    G.remove_edges_from(edges_to_remove)
    
    G.add_edges_from(add_edge_stat.keys())
    nx.set_edge_attributes(G, add_edge_stat)
    
    for page_nodes in nodes_to_add:
        G.add_node(page_nodes, pos=nodes_to_add[page_nodes][page_nodes]['pos'])
        page_node_attrs = {page_nodes: {'rpos':nodes_to_add[page_nodes][page_nodes]['rpos'], 'phase': nodes_to_add[page_nodes][page_nodes]['phase'], 'type':nodes_to_add[page_nodes][page_nodes]['type'], 'size': nodes_to_add[page_nodes][page_nodes]['size'], 'range': nodes_to_add[page_nodes][page_nodes]['range']}}
        nx.set_node_attributes(G, page_node_attrs)
        # print added new node
        print(f"add new node: {page_nodes} -> {G.nodes[page_nodes]}")
    return G

if ADD_ADDR:
    add_edge_stat,edges_to_remove,nodes_to_add = f2g.add_file_page(G_VFD, file_page_nodes_attr, dset_page_edges)
    G_VFD = update_nodes_edges(G_VFD,add_edge_stat, edges_to_remove, nodes_to_add)

    print(f"edges_to_remove: {edges_to_remove}")

add new node: [0-76)-2-R -> {'pos': (16, 0), 'rpos': 1, 'phase': 2, 'type': 'addr', 'size': 4980736, 'range': (0, 76)}
add new node: [76-152)-2-R -> {'pos': (16, 1), 'rpos': 1, 'phase': 2, 'type': 'addr', 'size': 4980736, 'range': (76, 152)}
add new node: [152-228)-2-R -> {'pos': (16, 2), 'rpos': 1, 'phase': 2, 'type': 'addr', 'size': 4980736, 'range': (152, 228)}
add new node: [0-76)-2-W -> {'pos': (14, 3), 'rpos': 1, 'phase': 2, 'type': 'addr', 'size': 4980736, 'range': (0, 76)}
add new node: [0-76)-3-W -> {'pos': (20, 0), 'rpos': 1, 'phase': 3, 'type': 'addr', 'size': 4980736, 'range': (0, 76)}
add new node: [0-76)-3-R -> {'pos': (22, 1), 'rpos': 1, 'phase': 3, 'type': 'addr', 'size': 4980736, 'range': (0, 76)}
add new node: [0-76)-4-R -> {'pos': (28, 0), 'rpos': 1, 'phase': 4, 'type': 'addr', 'size': 4980736, 'range': (0, 76)}
add new node: [152-228)-4-R -> {'pos': (28, 1), 'rpos': 1, 'phase': 4, 'type': 'addr', 'size': 4980736, 'range': (152, 228)}
add new node: [76-152)-4-R -> {'

In [9]:
# G_VFD = vfd2g.set_task_position_full(G_VFD, task_file_map, STAGE_START)
# sp.display_all_nodes_attr(G_VFD)
all_edge_types = nx.get_edge_attributes(G_VFD,'edge_type')
edge_types = []
for edge in all_edge_types:
    edge_types.append(all_edge_types[edge])

print(f"edge_types: {set(edge_types)}")

# sp.draw_graph(G_VFD, test_name, stat_path, graph_type="vfd", prefix=f'{(STAGE_END+1)}s', save=True)

edge_types: {'page-dset', 'task-dset', 'dset-task', 'dset-page', 'file-page', 'page-file'}


In [10]:
# sp.display_all_edges_attr(G_VFD)
# sp.display_all_nodes_position(G_VFD)

# Add networkx to Sankey diagram

## Statistics for Sankey
Below are needed edge attributes before generating the sankey diagram:
- access_cnt : The total file/dataset access count 
- access_size : The total read and write access size
- operation : The operation type : read, write, read_write
- bandwidth : Get the per access size and time, then calculate the bandwidth. Average the bandwidth if multiple accesses. (TODO: currently only recording one access size and time)




In [11]:
if ADD_ADDR: f2g.prepare_sankey_stat_full(G_VFD, vol_dict)
else: f2g.prepare_sankey_stat_no_addr(G_VFD, vol_dict)
    
vfd2sk.time_to_file_x_pos(G_VFD)

edge: ('run_gettracks-71911', 'basetime_new-2-W') -> task-dset
edge: ('run_gettracks-71911', 'basetime_ref-2-W') -> task-dset
edge: ('run_gettracks-71911', 'file-2-W') -> task-dset
edge: ('run_gettracks-71911', 'nclouds_new-2-W') -> task-dset
edge: ('run_gettracks-71911', 'nclouds_ref-2-W') -> task-dset
edge: ('run_gettracks-71911', 'newcloud_backward_index-2-W') -> task-dset
edge: ('run_gettracks-71911', 'newcloud_backward_size-2-W') -> task-dset
edge: ('run_gettracks-71911', 'nlinks-2-W') -> task-dset
edge: ('run_gettracks-71911', 'refcloud_forward_index-2-W') -> task-dset
edge: ('run_gettracks-71911', 'refcloud_forward_size-2-W') -> task-dset
edge: ('run_gettracks-71911', 'time-2-W') -> task-dset
edge: ('run_gettracks-71911', '-2-W') -> task-dset
edge: ('run_gettracks-71911', 'basetimes-2-W') -> task-dset
edge: ('run_gettracks-71911', 'cloudid_files-2-W') -> task-dset
edge: ('run_gettracks-71911', 'ncharacters-2-W') -> task-dset
edge: ('run_gettracks-71911', 'nclouds-2-W') -> task-d

In [12]:
stat_str = sp.show_vfd_stats(G_VFD)
print(stat_str)

# Count the number of nodes and edges
print(f"Number of nodes: {G_VFD.number_of_nodes()}")
print(f"Number of edges: {G_VFD.number_of_edges()}")

Total number of links: 4255
Total I/O size: 7.33 GB
Total I/O count: 375502
Total bandwidth: 661.89 GB/s
Average I/O size: 19.51 KB
Medium I/O size: 64 B
Inital input size: 2.88 KB

Number of nodes: 1157
Number of edges: 4255


In [13]:
import time
num_nodes = len(G_VFD.nodes)
num_edges = len(G_VFD.edges)

start_time = time.time()
vfd_nodes, vfd_nodes_dict = vfd2sk.get_nodes_for_sankey(G_VFD, rm_tags=["-R", "-W", "PID", "", ""], label_on=True)
print(f"Time to get [{num_nodes}] nodes for sankey: {time.time() - start_time}")
# print("--- get[]_nodes_for_sankey: %s seconds ---" % (time.time() - start_time))
# print(vfd_nodes)

start_time = time.time()
vfd_links = vfd2sk.parallel_get_links_for_sankey(G_VFD, vfd_nodes_dict, val_sqrt=True, highlight=False, edge_bw=False)
fig = go.Figure(go.Sankey(
            node = vfd_nodes,
            link = vfd_links, orientation='h'))
print(f"Time to get links [{num_edges}] for sankey: {time.time() - start_time}")
# print("--- get_links_for_sankey: %s seconds ---" % (time.time() - start_time))

width = 1200
height = 1000

fig.update_layout(
    autosize=False,
    width=width,
    height=height,
    margin=dict(
        l=width/10,
        r=width/1.5,
        b=height/2,
        t=height/4,
        # pad=2
    ),
    font=dict(size=18),
)


# fig.show()
start_time = time.time()
save_html_path = f"{stat_path}/vfd-{(STAGE_END+1)}s-{test_name}-sankey-labeled-s4.html"
fig.write_html(save_html_path)
print(f"Sankey saved to {save_html_path}")
print("--- write_html: %s seconds ---" % (time.time() - start_time))

Time to get [1157] nodes for sankey: 0.004418134689331055
Time to get links [4255] for sankey: 1.1910006999969482
Sankey saved to example_stat/s9f9p8_0/vfd-9s-s9f9p8_0-sankey-labeled-s4.html
--- write_html: 0.0829164981842041 seconds ---


In [14]:
import math
import networkx as nx
from concurrent.futures import ThreadPoolExecutor
from functools import partial
from scipy.stats import rankdata
import time

EDGE_COLOR_RGBA = {
    'none' : {'r':180, 'g':180, 'b':180}, #grey for open/close/meta
    'read_only' : {'r':150, 'g':190, 'b':220},
    'write_only' : {'r':150, 'g':190, 'b':220},
    'read_write' : {'r':150, 'g':190, 'b':220}, # TODO: currently showing same as write
    'highlight' : {'r':255, 'g':128, 'b':0}, # Orange ('r':255, 'g':128, 'b':0)
    # 'read_write' : {'r':220, 'g':220, 'b':220},
}

# color names : https://www.w3schools.com/colors/colors_names.asp
COLOR_MAP = {"task": "Red", # read
            "dataset": "Gold", # yellow
            "group/attr": "Yellow", # 
            "file": "MediumBlue", # blue
            "addr": "RoyalBlue", # slightly darker blue than file
            "none": "grey",
            }

OPACITY = 0.2

def edge_color_scale(G, node, attr_bw, attr_op, bw, op, highlight_on):
    node_attr = G.nodes[node]
    edge_highlight = False
    
    if highlight_on and node_attr['type'] == 'file':
            if G.out_degree(node) > 1:
                edge_highlight = True

    range = 100

    base_color_dict = {}
    if edge_highlight:
        # print(f"Node {node} has more than 2 out edges (will be highlighted)")
        base_color_dict = EDGE_COLOR_RGBA['highlight']
        r = base_color_dict['r']
        g = base_color_dict['g']
        b = base_color_dict['b']
        # range = 50
        color_str = f"rgba({r}, {g}, {b}, {OPACITY})"
        return color_str
    elif op in EDGE_COLOR_RGBA.keys():
        base_color_dict = EDGE_COLOR_RGBA[op]
        r = base_color_dict['r']
        g = base_color_dict['g']
        b = base_color_dict['b']
    else:
        base_color_dict = EDGE_COLOR_RGBA['none']
        r = base_color_dict['r']
        g = base_color_dict['g']
        b = base_color_dict['b']
        color_str = f"rgba({r}, {g}, {b}, {OPACITY})"
        return color_str
    
    

    edges = []
    # for k,v in attr_op.items():
    #     if v == op:
    #         edges.append(k)
    for op in attr_op:
        if attr_op[op] == 'read_write':
            edges.append(op)
    
    # bw_list = list(set(attr_bw.values()))
    bw_list = [attr_bw[x] for x in edges]
    bw_list.sort()
    # print(bw_list)
        
    color_ranks = rankdata(bw_list,method='dense')
    color_ranks = [float(i)/max(color_ranks) for i in color_ranks] # normalize

    my_rank = color_ranks[bw_list.index(bw)]

    color_change = my_rank *range #(my_rank-1)
    op_change = (my_rank/max(color_ranks)) * 0.5 + 0.2
    color_str = f"rgba({r-color_change/1.5}, {g-color_change/1.5}, {b-color_change/1.5}, {op_change})"

    # print(f"{color_str} rank={my_rank} sqrt={math.sqrt(my_rank)} change={color_change}")
    return color_str




In [15]:
# # "PID", "DATASETNAME", "FILENAME"
# attr_bw = nx.get_node_attributes(G_VFD,'bandwidth')
# print(f"bandwidth range: {sp.humanbw(min(attr_bw.values()))} ~ {sp.humanbw(max(attr_bw.values()))}")
    

# import time
# start_time = time.time()
# vfd_nodes, vfd_nodes_dict = vfd2sk.get_nodes_for_sankey(G_VFD, rm_tags=["-R", "-W", "PID", "", ""], label_on=True)
# print("--- get_nodes_for_sankey: %s seconds ---" % (time.time() - start_time))
# # print(vfd_nodes)

# start_time = time.time()
# vfd_links = vfd2sk.parallel_get_links_for_sankey(G_VFD, vfd_nodes_dict, val_sqrt=True, highlight=False, edge_bw=False)
# fig = go.Figure(go.Sankey(
#             node = vfd_nodes,
#             link = vfd_links, orientation='h'))
# print("--- get_links_for_sankey: %s seconds ---" % (time.time() - start_time))

# width = 1200
# height = 1000

# fig.update_layout(
#     autosize=False,
#     width=width,
#     height=height,
#     margin=dict(
#         l=width/10,
#         r=width/1.5,
#         b=height/2,
#         t=height/4,
#         # pad=2
#     ),
#     font=dict(size=18),
# )


# # fig.show()
# start_time = time.time()
# save_html_path = f"{stat_path}/vfd-{(STAGE_END+1)}s-{test_name}-sankey-labeled-s4.html"
# fig.write_html(save_html_path)
# print(f"Sankey saved to {save_html_path}")
# print("--- write_html: %s seconds ---" % (time.time() - start_time))

In [16]:
# Create subgraph

def remove_graph_after_node(G, node):
    stack = [node]  # Use a stack to simulate recursion

    while stack:
        current_node = stack.pop()
        out_edges = list(G.out_edges(current_node))
        # if out edges match the passed in node, do not remove the edge
        out_edges = [edge for edge in out_edges if edge[1] != node]
        
        if not out_edges:
            continue
        for edge in out_edges:
            downstream_node = edge[1]
            stack.append(downstream_node)

        G.remove_edges_from(out_edges)

    # Remove disconnected nodes
    for node in list(nx.isolates(G)):
        G.remove_node(node)
    
    for node in new_G.nodes:
        nodes_to_remove = []
        # remove node contain string "embedding"
        if "embedding" in node:
            nodes_to_remove.append(node)
    G.remove_nodes_from(nodes_to_remove)



In [17]:
if SUBGRAPH:
    new_G = G_VFD.copy()
    after_node="flintstones_out.h5" #"flintstones_out.h5" "training-42584" # exact name
    remove_graph_after_node(new_G, after_node)
    print(f"Is G_VFD and new_G the same: {nx.is_isomorphic(G_VFD, new_G)}")

    vfd_nodes, vfd_nodes_dict = vfd2sk.get_nodes_for_sankey(new_G, rm_tags=["-R", "-W", "PID", ], label_on=True) # "-1", "-2", "-3"

    # print(vfd_nodes)

    vfd_links = vfd2sk.get_links_for_sankey(new_G, vfd_nodes_dict, val_sqrt=True, highlight=True)
    fig = go.Figure(go.Sankey(
                node = vfd_nodes,
                link = vfd_links, orientation='h'))

    width = 1000
    height = 1000

    fig.update_layout(
        autosize=False,
        width=width,
        height=height,
        margin=dict(
            l=width/10,
            r=width/10,
            b=height/10,
            t=height/4,
            # pad=2
        ),
        font=dict(size=18),
    )

    fig.show()
    save_html_path = f"{stat_path}/vfd-{(STAGE_END+1)}s-{test_name}-sankey-labeled-s4-subgraph.html"
    fig.write_html(save_html_path)
    print(f"Sankey saved to {save_html_path}")

In [ ]:
# # Create a manual legend using annotations.

# def add_legend_to_graph(fig,save_img=False):
#     LEGEND_ITEMS = {
#         "Tasks":{"color":"red", "text":"Tasks"},
#         "Files":{"color":"blue", "text":"Files"},
#         "Edges":{"color":"lightblue", "text":"File bandwidth, darker the color, higher the bandwidth"},
#     }

#     legend_items = [
#         go.layout.Annotation(
#             x=0.9,  # X-coordinate for legend item
#             y=0.85 - i * 0.03,  # Y-coordinate for legend item (adjust for position)
#             xref='paper',
#             yref='paper',
#             text=f"{item_type} - {LEGEND_ITEMS[item_type]['color']}",
#             showarrow=False,
#             font=dict(size=14, color=LEGEND_ITEMS[item_type]['color']),
#             # bordercolor='grey',  # Set the border color for the legend box
#             # borderwidth=1,  # Set the border width for the legend box
#             bgcolor='rgba(255, 255, 255, 0.7)',  # Add a transparent background color
#         )
#         for i, item_type in enumerate(LEGEND_ITEMS.keys())
#     ]
#     fig.update_layout(annotations=legend_items)
#     fig.show()
#     if save_img:
#         fig.write_html(f"{stat_path}/vfd-{(STAGE_END+1)}s-{test_name}-sankey-annotated.html")
